In [1]:
from IPython.display import clear_output
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import re
import dropbox
import sys
import pandas as pd
import numpy as np
import imageio.v2 as imageio
import matplotlib.pyplot as plt
import cv2
import logging
import datetime
import glob
import json
import scipy
import matplotlib as mpl

from subprocess import call
from tifffile import imwrite
from tqdm import tqdm
from pathlib import Path
from amftrack.util.dbx import (
    upload_folder,
    download,
    read_saved_dropbox_state,
    save_dropbox_state,
    load_dbx,
    get_dropbox_folders_prince,
    get_dropbox_video_folders,
    download_video_folders_drop,
    download_analysis_folders_drop,
)
from amftrack.pipeline.launching.run import (
    run_transfer,
)
from amftrack.pipeline.launching.run_super import run_parallel_transfer
from amftrack.pipeline.launching.run_super import run_parallel_flows
from amftrack.pipeline.functions.transport_processing.high_mag_videos.plot_data import (
    plot_summary,
    save_raw_data,
)
from amftrack.pipeline.functions.transport_processing.high_mag_videos.high_mag_analysis import (
    HighmagDataset,
    VideoDataset,
    EdgeDataset,
    index_videos_dropbox_new,
    analysis_run,
)
from amftrack.pipeline.functions.transport_processing.high_mag_videos.kymo_class import (
    KymoVideoAnalysis,
    KymoEdgeAnalysis,
)
from IPython.display import clear_output

%matplotlib widget
%load_ext autoreload
%autoreload 2
logging.basicConfig(stream=sys.stdout, level=logging.debug)
mpl.rcParams["figure.dpi"] = 200

/gpfs/home2/cbisot/pycode/AMFtrack/amftrack/util/dbx.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
2024-05-27 08:39:28.342038: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-27 08:39:28.454232: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-27 08:39:28.745495: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-27 08:39:31.931945: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


2024-05-27 08:39:37,791-[WARNING]- absl:256 -> Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
2024-05-27 08:39:37,804-[WARNING]- absl:184 -> Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.


/home/cbisot/miniconda3/envs/amftrack/lib/python3.12/site-packages/keras/src/optimizers/base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


## File declaration
As this notebook is designed to work with Snellius (now also on a local computer!), two items to separate are the raw video files and the analysis. The raw video files are large, bulky and not so easy to flip through. Ideally, the video files would be downloaded and the analysis would be stored on a separate folder structure entirely. That way, large scale analysis of analysis folders can happen when there are thousands of videos in the dataset, without having to have those raw video folders on hand.

Below function will basically make your folders fertile ground to accept all the video info folders and raw video files.

### Input:
Please give separately the folder where raw video data is stored, and where the analysis will be stored. Also give the dropbox address of the dataset you want to analyze.

### Output:
The specified dropbox folder will be looked through, and all relevant video information will be downloaded to an analysis folder structure identical to what is present on teh dropbox. The relevant raw video folder structure will also be generated, if specified so. Will also create cache files in the form of .json files such that next time, the scrounging does not have to happen.

In [2]:
# videos_folder = "F:\\AMOLF_Data\\videos\\"
# analysis_folder = "F:\\AMOLF_Data\\analysis\\"

# videos_folder = "/gpfs/scratch1/shared/amftrackflow/videos/"
# analysis_folder = "/gpfs/home6/svstaalduine/Analysis/"
videos_folder = "/projects/0/einf914/videos/"

analysis_folder = "/projects/0/einf914/analysis_videos/"

In [3]:
# dropbox_address = "/DATA/FLUORESCENCE/DATA_NileRed/"
# dropbox_address=  "/DATA/MYRISTATE/DATA/2_weeks/"
# dropbox_address = "/DATA/TransportROOT/DATA/"
# dropbox_address = "/DATA/MYRISTATE/MorrisonDATA/20230508_Plate067/"
dropbox_address = "/DATA/CocoTransport/"
# dropbox_address = "/DATA/CocoCut/"

# dropbox_address = "/DATA/MYRISTATE/MorrisonDATA/"

# dropbox_address = "/DATA/TRANSPORT/DATA/20230308_Plate070/"

In [4]:
# names = [
#     "20230901_Plate310",
#     "20230902_Plate310",
#     "20230903_Plate310",
#     "20230904_Plate310",
#     "20230905_Plate310",
#     "20230906_Plate310",
# ]
# names += [
#     "20230810_Plate441",
#     "20230811_Plate441",
#     "20230812_Plate441",
#     "20230813_Plate441",
# ]
names = [
    "20230813_Plate449",
    "20230814_Plate449",
    "20230815_Plate449",
    "20230816_Plate449",
    "20230818_Plate449",
]
names += [
    "20230810_Plate441",
    "20230811_Plate441",
    "20230812_Plate441",
    "20230813_Plate441",
]

In [5]:
for name in names:
    video_param_frame = index_videos_dropbox_new(
        analysis_folder,
        videos_folder,
        f"{dropbox_address}{name}/",
        REDO_SCROUNGING=True,
        # date_start=20230801,
        # date_end=20230813,
        plate_names=None,
    )

Redoing the dropbox scrounging, hold on tight.
/DATA/CocoTransport/20230811_Plate441
20230811 Plate441
/DATA/CocoTransport/20230811_Plate441
2024-04-24 14:23:00,338-[INFO]- dropbox:390 -> Refreshing access token.
2024-04-24 14:23:00,600-[INFO]- dropbox:474 -> Request to files/list_folder
2024-04-24 14:23:01,084-[INFO]- dropbox:474 -> Request to files/get_metadata
2024-04-24 14:23:01,495-[INFO]- dropbox:474 -> Request to files/get_metadata
2024-04-24 14:23:01,747-[INFO]- dropbox:474 -> Request to files/get_metadata
2024-04-24 14:23:02,016-[INFO]- dropbox:474 -> Request to files/get_metadata
2024-04-24 14:23:02,278-[INFO]- dropbox:474 -> Request to files/get_metadata
2024-04-24 14:23:02,536-[INFO]- dropbox:474 -> Request to files/get_metadata
2024-04-24 14:23:02,876-[INFO]- dropbox:474 -> Request to files/get_metadata
2024-04-24 14:23:03,119-[INFO]- dropbox:474 -> Request to files/get_metadata
2024-04-24 14:23:03,405-[INFO]- dropbox:474 -> Request to files/get_metadata
2024-04-24 14:23:0

100%|████████████████████████████████████████████████████████████████████████████████| 196/196 [00:01<00:00, 130.58it/s]


In [6]:
video_param_frame[video_param_frame['magnification']==4]

,imaging_day,storage_path,plate_id,root,strain,treatment,crossing_day,video_int,time_(s),mode,...,ypos,zpos,unique_id,folder,tot_path_drop,record_time,days_after_crossing,magnification,analysis_folder,videos_folder
0,20230811,Dropbox\DATA\CocoTransport,20230811_Plate441,Carrot,C2,001P100N100C,20230807,1,30.0,F,...,1.810,13.319,20230811_Plate441_001,CocoTransport/20230811_Plate441/001/Img,DATA/CocoTransport/20230811_Plate441/001/Img,17:08:01,4,4.0,NaN,NaN
1,20230811,Dropbox\DATA\CocoTransport,20230811_Plate441,Carrot,C2,001P100N100C,20230807,2,20.0,BF,...,2.119,13.301,20230811_Plate441_002,CocoTransport/20230811_Plate441/002/Img,DATA/CocoTransport/20230811_Plate441/002/Img,17:10:17,4,4.0,NaN,NaN
2,20230811,Dropbox\DATA\CocoTransport,20230811_Plate441,Carrot,C2,001P100N100C,20230807,3,20.0,F,...,2.119,13.301,20230811_Plate441_003,CocoTransport/20230811_Plate441/003/Img,DATA/CocoTransport/20230811_Plate441/003/Img,17:10:55,4,4.0,NaN,NaN
3,20230811,Dropbox\DATA\CocoTransport,20230811_Plate441,Carrot,C2,001P100N100C,20230807,5,20.0,BF,...,0.536,13.236,20230811_Plate441_005,CocoTransport/20230811_Plate441/005/Img,DATA/CocoTransport/20230811_Plate441/005/Img,17:13:58,4,4.0,NaN,NaN
4,20230811,Dropbox\DATA\CocoTransport,20230811_Plate441,Carrot,C2,001P100N100C,20230807,6,20.0,F,...,0.536,13.236,20230811_Plate441_006,CocoTransport/20230811_Plate441/006/Img,DATA/CocoTransport/20230811_Plate441/006/Img,17:16:30,4,4.0,NaN,NaN
5,20230811,Dropbox\DATA\CocoTransport,20230811_Plate441,Carrot,C2,001P100N100C,20230807,7,10.0,BF,...,-1.494,13.271,20230811_Plate441_007,CocoTransport/20230811_Plate441/007/Img,DATA/CocoTransport/20230811_Plate441/007/Img,17:19:17,4,4.0,NaN,NaN
6,20230811,Dropbox\DATA\CocoTransport,20230811_Plate441,Carrot,C2,001P100N100C,20230807,8,10.0,F,...,-1.494,13.271,20230811_Plate441_008,CocoTransport/20230811_Plate441/008/Img,DATA/CocoTransport/20230811_Plate441/008/Img,17:19:43,4,4.0,NaN,NaN
7,20230811,Dropbox\DATA\CocoTransport,20230811_Plate441,Carrot,C2,001P100N100C,20230807,9,10.0,BF,...,-4.609,13.252,20230811_Plate441_009,CocoTransport/20230811_Plate441/009/Img,DATA/CocoTransport/20230811_Plate441/009/Img,17:21:17,4,4.0,NaN,NaN
8,20230811,Dropbox\DATA\CocoTransport,20230811_Plate441,Carrot,C2,001P100N100C,20230807,10,20.0,F,...,-4.609,13.252,20230811_Plate441_010,CocoTransport/20230811_Plate441/010/Img,DATA/CocoTransport/20230811_Plate441/010/Img,17:21:51,4,4.0,NaN,NaN
9,20230811,Dropbox\DATA\CocoTransport,20230811_Plate441,Carrot,C2,001P100N100C,20230807,11,10.0,BF,...,-6.129,13.311,20230811_Plate441_011,CocoTransport/20230811_Plate441/011/Img,DATA/CocoTransport/20230811_Plate441/011/Img,17:22:45,4,4.0,NaN,NaN


***To delete for 441***

20230809_1806_Plate14

20230809_2005_Plate14

20230809_2205_Plate14

20230812_0004_Plate14

20230812_1618_Plate14

20230813_2230_Plate14



***To delete for 449***

20230814_1218_Plate10

20230813_2219_Plate10

20230816_1227_Plate10

20230816_1628_Plate10

20230818_1307_Plate10

20230818_1523_Plate10



In [5]:
for name in names:
    video_param_frame = index_videos_dropbox_new(
        analysis_folder,
        videos_folder,
        f"{dropbox_address}{name}/",
        REDO_SCROUNGING=True,
        # date_start=20230801,
        # date_end=20230813,
        plate_names=None,
    )
    download_frame = video_param_frame.copy()
    run_parallel_transfer(
        "from_drop_video.py",
        [videos_folder],
        download_frame,
        20,
        "24:00:00",
        "transfer_test",
    )
    clear_output(wait=False)

clear_output(wait=True)

# Module 2: Processing

Now that the files have been downloaded, it's time to analyse them. In the below code, you'll be able to either do a complete survey of the analysis folder for as many videos as possible, or use the DataFrame of recently downloaded videos to filter for the videos you want to analyse.

Also possible to analyse videos directly in this notebook. Be aware again that this is a sequential, and slower analysis than running a SLURM job. 

### Input:
DataFrame filters of all videos to be analysed
### Output:
Print statements for all parameters of the analysis session that is about to take place.

In [5]:
# For indexing analysis folders
for name in names:
    video_param_frame = index_videos_dropbox_new(
        analysis_folder,
        videos_folder,
        f"{dropbox_address}{name}/",
        REDO_SCROUNGING=True,
        # date_start=20230801,
        # date_end=20230813,
        plate_names=None,
    )
    clear_output(wait=False)

clear_output(wait=True)

Redoing the dropbox scrounging, hold on tight.
/DATA/CocoTransport/20230904_Plate310
20230904 Plate310
/DATA/CocoTransport/20230904_Plate310
2024-05-26 22:13:40,526-[INFO]- dropbox:390 -> Refreshing access token.
2024-05-26 22:13:40,718-[INFO]- dropbox:474 -> Request to files/list_folder
2024-05-26 22:13:41,141-[INFO]- dropbox:474 -> Request to files/get_metadata
2024-05-26 22:13:41,365-[INFO]- dropbox:474 -> Request to files/get_metadata
2024-05-26 22:13:41,670-[INFO]- dropbox:474 -> Request to files/get_metadata
2024-05-26 22:13:41,937-[INFO]- dropbox:474 -> Request to files/get_metadata
2024-05-26 22:13:42,266-[INFO]- dropbox:474 -> Request to files/get_metadata
2024-05-26 22:13:42,513-[INFO]- dropbox:474 -> Request to files/get_metadata
2024-05-26 22:13:42,776-[INFO]- dropbox:474 -> Request to files/get_metadata
2024-05-26 22:13:43,004-[INFO]- dropbox:474 -> Request to files/get_metadata
2024-05-26 22:13:43,245-[INFO]- dropbox:474 -> Request to files/get_metadata
2024-05-26 22:13:4

  0%|                                                                                           | 0/132 [00:00<?, ?it/s]

2024-05-26 22:14:19,100-[INFO]- dropbox:390 -> Refreshing access token.
2024-05-26 22:14:19,295-[INFO]- dropbox:474 -> Request to files/download


  1%|▋                                                                                  | 1/132 [00:00<01:25,  1.53it/s]

2024-05-26 22:14:19,745-[INFO]- dropbox:390 -> Refreshing access token.
2024-05-26 22:14:19,958-[INFO]- dropbox:474 -> Request to files/download


...

 86%|█████████████████████████████████████████████████████████████████████▎           | 113/132 [01:25<00:14,  1.28it/s]

2024-05-26 22:15:44,789-[INFO]- dropbox:390 -> Refreshing access token.
2024-05-26 22:15:44,990-[INFO]- dropbox:474 -> Request to files/download


 86%|█████████████████████████████████████████████████████████████████████▉           | 114/132 [01:26<00:14,  1.26it/s]

2024-05-26 22:15:45,602-[INFO]- dropbox:390 -> Refreshing access token.


 86%|█████████████████████████████████████████████████████████████████████▉           | 114/132 [01:26<00:13,  1.32it/s]

KeyboardInterrupt



In [5]:
folder_filter = dropbox_address[5:]

img_infos = glob.glob(
    f"{analysis_folder}{folder_filter}/**/video_data.json", recursive=True
)
vid_anls_frame = pd.DataFrame()
for address in img_infos:
    add_info = pd.read_json(address, orient="index").T
    vid_anls_frame = pd.concat([vid_anls_frame, add_info], ignore_index=True)

vid_anls_frame = vid_anls_frame.sort_values("unique_id").reset_index(drop=True)
# vid_anls_frame.tail(20)

In [6]:
####################################################################################
### This is where you can apply the filters. Only those videos will be analyzed. ###
####################################################################################

# analysis_frame = vid_anls_frame[
#     vid_anls_frame["imaging_day"].ge("20230814")
# ].reset_index(drop=True)
# analysis_frame = vid_anls_frame[vid_anls_frame['xpos'].le(100)].reset_index(drop=True)
# analysis_frame = analysis_frame[analysis_frame['mode']=="F"]
# analysis_frame = vid_anls_frame[vid_anls_frame['plate_id'] != "20230729_Plate440"]
# analysis_frame = analysis_frame[analysis_frame['video_int'].isin([1])]
analysis_frame = vid_anls_frame
analysis_frame = analysis_frame.loc[analysis_frame["plate_id"].isin(names)]
####################################################################################
### Below code will prepare for those videos to be downloaded to videos_folder.  ###
####################################################################################

print(f"Number of videos to be analyzed: {len(analysis_frame)}")
# analysis_frame.tail(20)

Number of videos to be analyzed: 1357


In [ ]:
analysis_frame["plate_id"].unique()

## Run SLURM Analysis job
Two options: For small analysis, use the first block. This will just do the calculations on the machine. For large-scale analysis, use the second block, as it will create a Snellius job.
## Input:
Snellius job parameters
## Output:
Analysis folder will be populated with analysis tiffs and csv sheets. At the same time, this analysis folder will also be uploaded to the dropbox.

In [7]:
### LARGE VIDEO ANALYSIS

nr_parallel = np.min([len(analysis_frame.index), 2])

run_parallel_flows(
    "flux_extract.py",
    [analysis_folder, 9, 0.95, 0.005, 200, dropbox_address],
    analysis_frame,
    nr_parallel,
    "2:00:00",
    "flux_extract",
    node="fat_rome",
    name_job="transport",
)
clear_output(wait=False)

print(
    "Sent all the jobs! Use the command '$ squeue' in the terminal to see the progress"
)

Sent all the jobs! Use the command '$ squeue' in the terminal to see the progress


In [8]:
nr_parallel = np.min([len(analysis_frame.index), 30])

run_parallel_transfer(
    "flux_upload.py",
    [analysis_folder, 9, 0.95, 0.005, 200, dropbox_address],
    analysis_frame,
    nr_parallel,
    "6:00:00",
    "flux_upload",
    node="staging",
    cpus=1,
    # dependency = "flux_extract.sh",
    name_job="flux_upload.sh",
)
clear_output(wait=False)

print(
    "Sent all the jobs! Use the command '$ squeue' in the terminal to see the progress"
)

Sent all the jobs! Use the command '$ squeue' in the terminal to see the progress


In [ ]:
"20230810_Plate441_001" in list(analysis_frame["unique_id"].astype(str))

In [ ]:
values_id = list(analysis_frame["unique_id"])
dbx = load_dbx()
img_infos = glob.glob(f"{analysis_folder}/**/video_data.json", recursive=True)
vid_anls_frame = pd.DataFrame()
for address in img_infos:
    if os.path.exists(address):
        add_info = pd.read_json(address, orient="index").T
        # print(add_info['unique_id'].iloc[0])
        if add_info["unique_id"].iloc[0] in values_id:
            plate_id_video = add_info["plate_id"].iloc[0]
            original_path = add_info["tot_path_drop"].iloc[0]

            # Replace the specific substring in the target path
            target_path = original_path.replace(
                f"/{plate_id_video}", f"/KymoSpeeDExtract/{plate_id_video}"
            )
            source = "/" + target_path + "/video_data_network.json"
            target = address.replace("video_data.json", "video_data_network.json")
            try:
                results = dbx.files_search(
                    "/" + target_path, "video_data_network.json"
                ).matches
                if results:
                    download(
                        source,
                        target,
                    )
            except dropbox.exceptions.ApiError:
                print(address)
    # break